# Advanced Quantum Algorithm Design
# Discrete-Time Quantum Walk on a Particular Graph
### from "Lectures Note on Quantum Algorithms" by <u>Andrew M. Childs</u>

Task: Design the quantum walk operator for the case of a line with 16 nodes.

<img src="./6.12 Graph.png" width="900">

Solution by <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/">`La Wun Nannda`</a>

# Solution

## 1. Prepare an Environment

In [1]:
# import libraries
from classiq import *
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# initiate a session
authenticate()

C:\Users\ASUS\anaconda3\envs\qiskit_env\Lib\site-packages\classiq\_internals\authentication\token_manager.py:99: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


## 2. Think about Graph

<img src="./6.12 Graph.png" width="900">

- In the given graph, we can see that except `vertex[0]` and `vertex[15]`, all other vertices have exactly two adjacent vertices.
- For each vertex, its adjacent vertices will be in superposition. This is how probability is distributed.
- For `vertex[0]` and `vertex[15]`, there is only one adjacent vertex with 100% probability.
- Since there are 16 vertices in total, we have to use 4 qubits to represent them as QNum.

In [3]:
# test function for probabilties for all vertices
def print_all_probs(size):
    '''
    # There is a `prob` for each vertex.
    # Let's say the first `prob` is for vertex "v".
    # The `len(prob)` corresponds to the number of all vertices including "v".
    # Inside `prob`, each value corresponds to the probability of other vertices being adjacent to "v" (graph dependent).
    '''
    for i in range(2**size): # for each vertex
        prob = [0,0,0,0, # initiated as zeros
                0,0,0,0,
                0,0,0,0,
                0,0,0,0]
        if i!=15: # for vertex[15], there is no vertex after it
            prob[(i+1)]=0.5
        if i==0: # for vertex[0], the P of vertex after it should be 1
            prob[(i+1)]+=0.5
        if i!=0: # for vertex[0], there is no vertex before it
            prob[(i-1)]=0.5
        if i==15: # for vertex[15], the P of vertex before it should be 1
            prob[(i-1)]+=0.5
        print(f'State={i}, prob vec ={prob}')

In [4]:
size = 4 # four qubits required
print_all_probs(size) # test getting probabilities

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0

We have successfully found our way with probabilities so we can finally implement a function for it.

In [5]:
# function for probabilities for a single vertex
def get_p(i):
    prob = [0,0,0,0, # initiated as zeros
            0,0,0,0,
            0,0,0,0,
            0,0,0,0]
    if i!=15: # for vertex[15], there is no vertex after it
        prob[(i+1)]=0.5
    if i==0: # for vertex[0], the P of vertex after it should be 1
        prob[(i+1)]+=0.5
    if i!=0: # for vertex[0], there is no vertex before it
        prob[(i-1)]=0.5
    if i==15: # for vertex[15], the P of vertex before it should be 1
        prob[(i-1)]+=0.5
    return prob

### 3. Prepare Functions
The main mathematical equations involved in discrete-time quantum walk are:
$$C:=\sum_{j\in V} |j\rangle\langle j| \otimes (2|\partial_j\rangle\langle\partial_j|-I)$$
$$S:=\sum_{(j,k)\in E}|j, k\rangle \langle k, j|$$

### 3.1. Functions on Phase Kickback
In discrete-time quantum walk, assume that $U_j|0\rangle=|\partial_j\rangle$. Then, $2|\partial_j\rangle\langle\partial_j|-1$ can be expanded as $U_j(2|0\rangle\langle0|-1)U'_j$. 

Within two unitaries is a phase kickback.
$$(2|0\rangle\langle0|-1)|0\rangle=|0\rangle~~ ----(1)$$
$$(2|0\rangle\langle0|-1)|j\neq0\rangle=-|j\neq0\rangle~~ ----(2)$$
From these two equations, we can define $f(x)$ such that it evaluates to
- `True` or `1` when $x\neq0$
- `False` or `0` if $x=0$

With that $f(x)$, our phase kickback function is defined: $$(2|0\rangle\langle0|-1)|x\rangle=(-1)^{f(x)}|x\rangle$$
which will evaluate to the same qubit with a coefficient of `1` when $x=0$ and `-1` for other $x$.

In [6]:
# prepare minus function for phase kickback
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)

In [7]:
# prepare oracle function for phase kickback
@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)

In [8]:
# phase kickback function
@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

### 3.2. Functions on the <i>C</i> Equation
The `C_operator` or `W_operator` has the following equation: 
$$C:=\sum_{j\in V} |j\rangle\langle j| \otimes (2|\partial_j\rangle\langle\partial_j|-I)~~ ----(1)$$
while 
$$|\partial_j\rangle:=\frac{1}{\sqrt{deg(j)}}\sum_{k:(j,k)\in E} |k\rangle~~ ----(2)$$

1. The first equation simply means that for every vertex "j", the phase kickback function is applied.
   - A part of it is implemented in above via the function called <i>zero_diffuzer()</i>.
3. The second one refers to the superposition of vertices "k" which are adjacent to the vertex "j".

In [9]:
# function implementing above two equations
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    # probabilities for graph
    prob = get_p(i)

    # the C_equation or W_equation
    control(ctrl=vertices==i, # the first equation # product of two "j" qubits
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices), # the second equation # partial differential
              action= lambda: zero_diffuzer(adjacent_vertices))) # the first equation # the remaining term after the tensor product

In [10]:
# repeat the above function for all "j" inside V
@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)

### 3.3. Functions on the <i>S</i> Equation
The following function checks an edge, i.e., whether two vertices are adjacent or not. It returns `True` if the vertex and its adjacent vertex have an edge between them. If the result is `True`, it performs a bitwise SWAP operation. Mathematically, the code is an implementation of
$$S:=\sum_{(j,k)\in E}|j, k\rangle \langle k, j|$$

In [11]:
# edge checker function
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) ==1) # depend on the graph

In [12]:
# function to swap if true
@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))

In [13]:
# put above two functions together
@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

## 4. Creation, Synthesis, and Circuit of the Quantum Model

In [14]:
# the main function
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

In [15]:
qmod = create_model(main) # create a model
qprog = synthesize(qmod) # synthesize it

In [16]:
show(qprog) # display the circuit

Opening: https://platform.classiq.io/circuit/e81aab8e-a6af-455d-9fcc-ed985f07ff5b?version=0.43.3


### 5. Conclusion
In conclusion, we have successfully implemented a discrete quantum walk algorithm for the given graph.

<img src="./6.12 Circuit.jpg">